In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import tensorflow as tf

# Define a dummy Cast layer if necessary
class CastLayer(Layer):
    def call(self, inputs):
        return tf.cast(inputs, tf.float32)

# Load model with custom objects
custom_objects = {"Cast": CastLayer}
model = load_model(r"C:\Users\HP\Documents\project\eye_disease_model1.keras", custom_objects=custom_objects)

# Print model summary
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,293,326 (4.93 MB)

 Trainable params: 430,788 (1.64 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 861,578 (3.29 MB)

None


In [4]:
import gradio as gr
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image  # Ensure you import this

# Load model
model = tf.keras.models.load_model(r"C:\Users\HP\Documents\project\eye_disease_model1.keras")

# Class labels
class_names = ["Cataract", "Diabetic Retinopathy", "Glaucoma", "Normal"]

# Treatment recommendations
treatment_recommendations = {
    "Cataract": "Surgery is the only effective treatment.",
    "Diabetic Retinopathy": "Strict blood sugar control and laser treatment may be needed.",
    "Glaucoma": "Medication, laser therapy, or surgery to reduce eye pressure.",
    "Normal": "No treatment needed, but regular checkups are recommended."
}

def predict(img):
    # Resize image to match model input
    img = img.resize((128, 128))
    img = np.array(img) / 255.0   # Normalize
    img = np.expand_dims(img, axis=0)

    # Model prediction
    prediction = model.predict(img)[0]

    # Get predicted class
    predicted_index = np.argmax(prediction)
    predicted_class = class_names[predicted_index]

    # Confidence scores
    confidence_scores = {class_names[i]: float(prediction[i]) for i in range(len(class_names))}

    # Get treatment recommendation
    recommendation = treatment_recommendations[predicted_class]

    return predicted_class, confidence_scores, recommendation

# Create Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Predicted Condition"),
        gr.Label(label="Confidence Scores"),
        gr.Textbox(label="Treatment Recommendation")
    ],
    title="Eye Disease Classifier",
    description="Upload an eye scan image to detect the condition and get a treatment recommendation."
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://3435fa9637befbe1e2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 676ms/step
